# Eddy Current Frequency Analysis
Characterizes braking dynamics via spectral decomposition

**Features**:  
- FFT power spectrum analysis  
- Resonance peak detection  
- Energy dissipation calculation  
- Comparative material analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
import os
import warnings
warnings.filterwarnings('ignore')

# Set matplotlib style for better plots
plt.style.use('default')
plt.rcParams['figure.dpi'] = 100

## 1. Load Dataset

In [ ]:
def load_data(filepath):
    """Load and preprocess sensor data from CSV file."""
    try:
        df = pd.read_csv(filepath)
        
        # Check if timestamp column exists
        if 'timestamp' not in df.columns:
            print(f"Warning: 'timestamp' column not found. Available columns: {df.columns.tolist()}")
            # Create synthetic timestamp if missing
            df['timestamp'] = np.arange(len(df)) * 10  # 10ms intervals
        
        t = df['timestamp'] / 1000  # Convert to seconds
        dt = np.mean(np.diff(t)) if len(t) > 1 else 0.01
        
        return df, t, dt
    except FileNotFoundError:
        print(f"Error: File {filepath} not found.")
        return None, None, None
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None

## 2. Spectral Analysis

In [ ]:
def compute_spectrum(signal, dt):
    """Compute power spectral density using FFT."""
    if len(signal) == 0 or dt <= 0:
        return np.array([]), np.array([])
    
    N = len(signal)
    # Remove DC component and apply window
    windowed_signal = signal - np.mean(signal)
    yf = fft(windowed_signal)
    xf = fftfreq(N, dt)[:N//2]
    
    # Compute power spectral density
    power = 2.0/N * np.abs(yf[0:N//2])
    
    return xf, power

## 3. Energy Dissipation Calculation

In [ ]:
def braking_energy(acceleration, mass, time):
    """Calculate kinetic energy from acceleration data."""
    if len(acceleration) == 0 or len(time) == 0:
        return np.array([])
    
    # Integrate acceleration to get velocity (assuming initial velocity = 0)
    dt = np.gradient(time)
    velocity = np.cumsum(acceleration * dt)
    
    # Calculate kinetic energy
    kinetic_energy = 0.5 * mass * velocity**2
    
    return kinetic_energy

## 4. Sample Data Generator (for testing)

In [ ]:
def generate_sample_data(filename, material_type="copper"):
    """Generate sample data for testing when real data files are not available."""
    
    # Create directories if they don't exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # Generate synthetic eddy current braking data
    t = np.linspace(0, 5000, 1000)  # 5 seconds, 1000 points (in ms)
    
    # Base exponential decay for braking
    base_signal = np.exp(-t/2000) * 10  # Exponential decay
    
    # Add material-specific characteristics
    if material_type == "copper":
        # Copper has higher conductivity, stronger braking
        noise_level = 0.1
        freq_content = 15  # Hz
    else:  # aluminum
        # Aluminum has lower conductivity, weaker braking
        base_signal *= 0.7
        noise_level = 0.15
        freq_content = 12  # Hz
    
    # Add some oscillatory content and noise
    sensor1 = base_signal + 0.5*np.sin(2*np.pi*freq_content*t/1000) + np.random.normal(0, noise_level, len(t))
    sensor2 = base_signal*0.9 + 0.3*np.cos(2*np.pi*(freq_content*1.2)*t/1000) + np.random.normal(0, noise_level, len(t))
    
    # Create DataFrame
    df = pd.DataFrame({
        'timestamp': t,
        'sensor1_filtered': sensor1,
        'sensor2_filtered': sensor2
    })
    
    df.to_csv(filename, index=False)
    print(f"Sample data generated: {filename}")

## 5. Main Analysis Workflow

In [ ]:
def analyze_run(filepath, material, thickness, magnet_strength):
    """Main analysis function for eddy current data."""
    # Load data with error handling
    df, t, dt = load_data(filepath)
    
    if df is None:
        print(f"Skipping analysis for {material} - data loading failed")
        return None
    
    mass = 0.5  # kg (car mass)
    
    # Check for required columns
    required_sensors = ['sensor1_filtered', 'sensor2_filtered']
    available_sensors = [col for col in required_sensors if col in df.columns]
    
    if not available_sensors:
        print(f"Warning: No filtered sensor columns found. Available columns: {df.columns.tolist()}")
        # Try to use raw sensor data
        available_sensors = [col for col in df.columns if 'sensor' in col.lower()]
        if not available_sensors:
            print("No sensor data found in dataset")
            return None
    
    fig, axs = plt.subplots(3, 1, figsize=(12, 10))
    
    # Time-domain plot
    colors = ['red', 'blue', 'green', 'orange']
    for i, sensor in enumerate(available_sensors[:2]):  # Limit to first 2 sensors
        if sensor in df.columns:
            axs[0].plot(t, df[sensor], colors[i], label=sensor, linewidth=1.5)
    
    axs[0].set_title(f'Time Domain: {material} {thickness}mm, {magnet_strength}T', fontsize=12)
    axs[0].set_ylabel('Acceleration (m/s²)')
    axs[0].legend()
    axs[0].grid(True, alpha=0.3)
    
    # Frequency-domain plot
    for i, sensor in enumerate(available_sensors[:2]):
        if sensor in df.columns:
            xf, yf = compute_spectrum(df[sensor], dt)
            
            if len(xf) > 0 and len(yf) > 0:
                axs[1].semilogy(xf, yf, color=colors[i], label=f'{sensor} PSD', linewidth=1.5)
                
                # Find dominant peaks
                if np.max(yf) > 0:  # Only if there's actual signal
                    peaks, properties = find_peaks(yf, prominence=np.max(yf)*0.1, height=np.max(yf)*0.05)
                    for idx in peaks[:3]:  # Top 3 peaks
                        if idx < len(xf) and idx < len(yf):
                            axs[1].annotate(f'{xf[idx]:.1f}Hz', 
                                           (xf[idx], yf[idx]),
                                           textcoords="offset points",
                                           xytext=(0,10), ha='center',
                                           fontsize=9)
    
    axs[1].set_title('Power Spectral Density')
    axs[1].set_xlabel('Frequency (Hz)')
    axs[1].set_ylabel('Power')
    axs[1].legend()
    axs[1].set_xlim(0, min(100, np.max(xf) if len(xf) > 0 else 50))
    axs[1].grid(True, alpha=0.3)
    
    # Energy plot
    for i, sensor in enumerate(available_sensors[:2]):
        if sensor in df.columns:
            # Convert acceleration to deceleration
            decel = df[sensor].values
            energy = braking_energy(decel, mass, t)
            
            if len(energy) > 0:
                axs[2].plot(t, energy, color=colors[i], label=f'{sensor} Energy', linewidth=1.5)
    
    axs[2].set_title('Kinetic Energy Dissipation')
    axs[2].set_xlabel('Time (s)')
    axs[2].set_ylabel('Energy (J)')
    axs[2].legend()
    axs[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save plot with error handling
    try:
        plt.savefig(f"{material}_analysis.png", dpi=150, bbox_inches='tight')
        print(f"Analysis plot saved as {material}_analysis.png")
    except Exception as e:
        print(f"Error saving plot: {e}")
    
    plt.show()
    return fig

## 6. Utility Functions

In [ ]:
def check_dependencies():
    """Check if all required packages are installed."""
    required_packages = ['numpy', 'pandas', 'matplotlib', 'scipy']
    missing_packages = []
    
    for package in required_packages:
        try:
            __import__(package)
        except ImportError:
            missing_packages.append(package)
    
    if missing_packages:
        print("❌ Missing packages:", missing_packages)
        print("Install with: pip install", " ".join(missing_packages))
    else:
        print("✅ All required packages are available!")
    
    return len(missing_packages) == 0

# Check dependencies
dependencies_ok = check_dependencies()

## 7. Example Usage

In [ ]:
if dependencies_ok:
    # Define materials and their properties
    materials = {
        "copper": ("data/sample_datasets/copper_plate.csv", 58.5, 2.0),
        "aluminum": ("data/sample_datasets/aluminum_braking.csv", 37.7, 2.0)
    }
    
    print("🔬 Starting Eddy Current Analysis...")
    print("=" * 50)
    
    results = {}
    
    for name, (path, sigma, thickness) in materials.items():
        print(f"\n📊 Analyzing {name.capitalize()} data...")
        
        # Generate sample data if file doesn't exist
        if not os.path.exists(path):
            print(f"📁 Data file {path} not found. Generating sample data...")
            generate_sample_data(path, name)
        
        # Perform analysis
        fig = analyze_run(path, name.capitalize(), thickness, 0.8)
        
        if fig is not None:
            results[name] = fig
            print(f"✅ Analysis completed for {name}")
        else:
            print(f"❌ Analysis failed for {name}")
    
    print("\n" + "=" * 50)
    print(f"🎉 Analysis workflow completed! Processed {len(results)}/{len(materials)} materials.")
else:
    print("❌ Cannot run analysis due to missing dependencies.")

## 8. Summary and Results

This notebook performs comprehensive eddy current frequency analysis including:

- **Time domain analysis**: Visualization of raw sensor data
- **Frequency domain analysis**: FFT-based power spectral density with peak detection
- **Energy analysis**: Kinetic energy dissipation calculations

The analysis compares different materials (copper vs aluminum) and their braking characteristics based on eddy current effects.